In [0]:
pip install surprise

In [0]:
import pandas as pd
from surprise import Reader, Dataset
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic
from surprise.model_selection import cross_validate
import time
import torch
from surprise import Dataset
from sklearn.model_selection import KFold
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

In [0]:
# load the dataset which contains 1 million movie ratings
data = Dataset.load_builtin('ml-1m')

## ***KNN***

In [0]:
start = time.time()

knn = KNNBasic()
cross_validate(knn, data, cv=5, verbose=True)

end = time.time()
print('Training time:',end-start)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9227  0.9241  0.9234  0.9231  0.9210  0.9229  0.0010  
MAE (testset)     0.7276  0.7285  0.7281  0.7274  0.7259  0.7275  0.0009  
Fit time          34.72   36.56   36.48   41.04   39.81   37.72   2.33    
Test time         129.87  131.59  135.84  135.34  134.87  133.50  2.35    
Training time: 867.935506105423


### **Singular Value Decomposition Matrix Factorization**

In [0]:
start = time.time()

svd = SVD()
cross_validate(svd, data, cv=5, verbose=True)

end = time.time()
print('Training time:',end-start)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8756  0.8752  0.8724  0.8710  0.8737  0.8736  0.0017  
MAE (testset)     0.6871  0.6872  0.6843  0.6837  0.6860  0.6856  0.0014  
Fit time          48.15   48.75   48.89   48.67   49.16   48.72   0.33    
Test time         2.71    2.54    2.63    2.57    2.09    2.51    0.22    
Training time: 268.19954109191895


### **Nongative Matrix Factorization**

In [0]:
start = time.time()

nmf = NMF()
cross_validate(nmf, data, cv=5, verbose=True)

end = time.time()
print('Training time:',end-start)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9168  0.9170  0.9150  0.9147  0.9169  0.9161  0.0010  
MAE (testset)     0.7247  0.7237  0.7224  0.7239  0.7249  0.7239  0.0009  
Fit time          52.46   52.34   52.58   52.75   52.21   52.47   0.19    
Test time         2.71    2.57    2.60    2.61    2.61    2.62    0.05    
Training time: 288.5303313732147


In [0]:
# fetch the raw data to turn it into a pandas dataframe object
ratings = data.raw_ratings
df = pd.DataFrame(ratings,columns = ['userId','movieId','rating','timestamp'])

In [0]:
df.head(10)

,userId,movieId,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291
5,1,1197,3.0,978302268
6,1,1287,5.0,978302039
7,1,2804,5.0,978300719
8,1,594,4.0,978302268
9,1,919,4.0,978301368


In [0]:
rating_matrix = df.pivot(index='userId', columns='movieId', values='rating')

In [0]:
rating_matrix.head(10)

movieId,1,10,100,1000,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,1034,...,960,961,962,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,98,980,981,982,984,985,986,987,988,989,99,990,991,992,993,994,996,997,998,999
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,3.0,NaN,NaN,3.0,5.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,5.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN,3.0,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,1.0,3.0,4.0,3.0,4.0,NaN,NaN,4.0,3.0,NaN,2.0,4.0,3.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Scaling rating to be between 0 and 1
min_rating, max_rating = df['rating'].min(), df['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)

# fill in -1 for missing items
rating_matrix[rating_matrix.isnull()] = -1

# get number of user and number of movies
n_users, n_movies = rating_matrix.shape

In [0]:
rating_matrix.head(10)

movieId,1,10,100,1000,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,1034,...,960,961,962,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,98,980,981,982,984,985,986,987,988,989,99,990,991,992,993,994,996,997,998,999
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000,-1.00,0.000,-1.0000,-1.000,-1.0000,-1.0,-1.0,0.0000,0.000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0
10,0.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,0.0000,-1.0,-1.0000,-1.0,-0.125,-1.0,-1.0,-0.125,0.0,-1.0,-1.0000,-0.0625,-1.0,-0.125,-1.00,0.000,0.0000,-1.000,-0.0625,-1.0,-1.0,0.0000,-1.000,-1.0,-0.1250,-0.0625,-0.0625,-1.0000,-1.0,...,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.000,-0.1250,-1.0,-1.0,0.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0
100,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000,-1.00,-1.000,-1.0000,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-0.0625,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0
1000,0.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000,-1.00,-1.000,0.0000,-1.000,-1.0000,-1.0,-1.0,-1.0000,-0.125,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0000,-1.0,-1.0,-0.0625,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0
1001,-0.0625,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000,-1.00,-1.000,-1.0000,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0
1002,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000,-1.00,-1.000,-1.0000,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.0625,-1.0,-1.0,-1.0,-1.0
1003,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0,-1.0000,-1.0,-1.0000,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000,-1.00,-1.000,-1.0000,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.0000,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0000,-1.0,-1.0,-1.0,-1.0
1004,0.0000,-0.1875,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.125,-1.0,-0.0625,-1.0,-0.0625,-1.0,-1.000,-1.0,-1.0,-1.000,-1.0,-1.0,-0.1875,-1.0000,-1.0,-0.125,-0.25,-0.125,-0.0625,-0.125,-0.0625,-1.0,-1.0,-0.0625,-0.125,-1.0,-0.1875,-0.0625,-0.1250,-0.0625,-

In [0]:
# convert to tensor
rating_matrix = torch.FloatTensor(rating_matrix.values)

In [0]:
print(type(rating_matrix))
print(rating_matrix.shape)

<class 'torch.Tensor'>
torch.Size([6040, 3706])


### ***Constrained Probalistic Matrix Factorization***

In [0]:
# customized loss function for constrained PMF
class PMF_lossfunc(torch.nn.Module):
  def __init__(self, lambda_u, lambda_v):
    super().__init__()
    self.lambda_u = lambda_u
    self.lambda_v = lambda_v

  def forward(self, rating_matrix, u_features, v_features):
    non_zero_mask = (rating_matrix != -1).type(torch.FloatTensor)
    predicted = torch.sigmoid(torch.mm(u_features, v_features.t()))
    
    diff = (rating_matrix - predicted)**2
    prediction_error = torch.sum(diff * non_zero_mask)

    # regularization 
    u_regul = self.lambda_u * torch.sum(u_features.norm(dim=1))
    v_regul = self.lambda_v * torch.sum(v_features.norm(dim=1))

    return prediction_error + u_regul + v_regul

In [0]:
predicted = torch.sigmoid(torch.mm(user_features, movie_features.t()))
predicted.shape

torch.Size([6040, 3706])

In [0]:
rating_matrix.shape

torch.Size([6040, 3706])

In [0]:
# Training constrained PMF
start = time.time()
latent_vectors = 100

user_features = torch.randn(n_users, latent_vectors, requires_grad= True)
user_features.data.mul(.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad = True)
user_features.data.mul(.01)

criterion = PMF_lossfunc(lambda_u=.002, lambda_v=.002)
optimizer = torch.optim.Adam([user_features, movie_features], lr=0.01)

for i, epoch in enumerate(range(2000)):
  optimizer.zero_grad()
  loss = criterion(rating_matrix, user_features, movie_features)
  loss.backward()
  optimizer.step()

  if i % 50 == 0:
    print(f"Epoch {i}, {loss:.4f}")

end = time.time()
print('Training time:',end-start)

Epoch 0, 309947.8438
Epoch 50, 211600.9844
Epoch 100, 171744.9531
Epoch 150, 136190.0000
Epoch 200, 104424.2031
Epoch 250, 78468.2109
Epoch 300, 59579.7539
Epoch 350, 46738.6914
Epoch 400, 37908.7539
Epoch 450, 31686.4395
Epoch 500, 27166.4355
Epoch 550, 23830.9727
Epoch 600, 21341.0469
Epoch 650, 19384.4004
Epoch 700, 17805.7227
Epoch 750, 16562.5918
Epoch 800, 15545.5234
Epoch 850, 14681.2676
Epoch 900, 13938.6914
Epoch 950, 13316.7119
Epoch 1000, 12767.8418
Epoch 1050, 12291.4111
Epoch 1100, 11865.7158
Epoch 1150, 11461.9795
Epoch 1200, 11110.9404
Epoch 1250, 10808.3281
Epoch 1300, 10523.5479
Epoch 1350, 10284.2461
Epoch 1400, 10056.3818
Epoch 1450, 9833.7959
Epoch 1500, 9641.2129
Epoch 1550, 9454.9961
Epoch 1600, 9280.3994
Epoch 1650, 9121.0254
Epoch 1700, 8978.2412
Epoch 1750, 8842.4326
Epoch 1800, 8720.9434
Epoch 1850, 8594.7178
Epoch 1900, 8474.6211
Epoch 1950, 8360.5625
Training time: 682.6559889316559


In [0]:
total_mse = 0
total_mae = 0
user_idx = 0

while user_idx < len(rating_matrix.numpy()):
  user_ratings = rating_matrix[user_idx, :]
  true_ratings = user_ratings != -1
  predictions = torch.sigmoid(torch.mm(user_features[user_idx, :].view(1, -1), movie_features.t()))
  predicted_ratings = (predictions.squeeze()[true_ratings]*(max_rating - min_rating) + min_rating).round()
  actual_ratings = (user_ratings[true_ratings]*(max_rating - min_rating) + min_rating).round()
  total_mse += mean_squared_error(predicted_ratings.detach().numpy(),actual_ratings.detach().numpy())
  total_mae += mean_absolute_error(predicted_ratings.detach().numpy(),actual_ratings.detach().numpy())
  user_idx += 1 

In [0]:
print("PMF RMSE: ")
print(math.sqrt(total_mse / (user_idx+1)))
print("PMF MAE:")
print(total_mae / (user_idx+1))

PMF RMSE: 
0.33026929566433844
PMF MAE:
0.06805832428261675
